## Chapter 7.函数
>本章节使用的模块：
1. html
2. functools
 - functools.partial
3. logging
4. multiprocessing
 - multiprocessing.Pool
5. math

#### 1.可接受任意数量参数的函数
使用带有*的参数

In [1]:
def avg(first, *rest):
    return (first + sum(rest)) / (1 + len(rest))

# Sample use
avg(1, 2, 3, 4) # 2.5

2.5

为了接受任意数量的关键字参数，使用一个以 **开头的参数。

In [ ]:
import html

def make_element(name, value, **attrs):
    keyvals = [' %s="%s"' % item for item in attrs.items()]
    attr_str = ''.join(keyvals)
    element = '<{name}{attrs}>{value}</{name}>'.format(
                name=name,
                attrs=attr_str,
                value=html.escape(value))
    return element

# Example
# Creates '<item size="large" quantity="6">Albatross</item>'
make_element('item', 'Albatross', size='large', quantity=6)

# Creates '<p>&lt;spam&gt;</p>'
make_element('p', '<spam>')

如果你还希望某个函数能同时接受任意数量的位置参数和关键字参数，可以同时使用*和**。

In [ ]:
def anyargs(*args, **kwargs):
    print(args) # A tuple
    print(kwargs) # A dict
    
#使用这个函数时，所有位置参数会被放到args元组中，所有关键字参数会被放到字典kwargs中。
#一个*参数只能出现在函数定义中最后一个位置参数后面，而 **参数只能出现在最后一个参数。

In [ ]:
def a(x, *args, y):
    pass

def b(x, *args, y, **kwargs):
    pass

#### 2. 只接受关键字参数的函数
你希望函数的某些参数强制使用关键字参数传递，将强制关键字参数放到某个*参数或者单个*后面就能达到这种效果。

In [ ]:
def recv(maxsize, *, block):
    'Receives a message'
    pass

recv(1024, True) # TypeError
recv(1024, block=True) # Ok

#--------很多情况下，使用强制关键字参数会比使用位置参数表意更加清晰，程序也更加具有可读性。
def minimum(*values, clip=None):
    m = min(values)
    if clip is not None:
        m = clip if clip > m else m
    return m

minimum(1, 5, 2, -5, 10) # Returns -5
minimum(1, 5, 2, -5, 10, clip=0) # Returns 0

#### 3. 给函数参数增加元信息
使用函数参数注解是一个很好的办法，它能提示程序员应该怎样正确使用这个函数:

In [1]:
def add(x:int, y:int) -> int:
    return x + y

help(add)
add.__annotations__
#python解释器不会对这些注解添加任何的语义。它们不会被类型检查，运行时跟没有加注解之前的效果也没有任何差距。
#函数注解只存储在函数的 __annotations__ 属性中。

Help on function add in module __main__:

add(x:int, y:int) -> int



{'return': int, 'x': int, 'y': int}

#### 4. 定义有默认参数的函数
默认参数在定义的时候已经确认，假如b=x，改变x的值也不会改变b的默认参数。

In [2]:
def spam(a, b=42):
    print(a, b)

spam(1) # Ok. a=1, b=42
spam(1, 2) # Ok. a=1, b=2

1 42
1 2


如果你并不想提供一个默认值，而是想仅仅测试下某个默认参数是不是有传递进来：  
仔细观察可以发现到传递一个None值和不传值两种情况是有差别的。

In [3]:
_no_value = object()

def spam(a, b=_no_value):
    if b is _no_value:
        print('No b value supplied')
    ...

spam(1)
spam(1, 2) # b = 2
spam(1, None) # b = None

No b value supplied


**一个函数需要测试某个可选参数是否被使用者传递进来：**  
这时候需要小心的是你不能用某个默认值比如None、 0或者False值来测试用户提供的值(因为这些值都是合法的值，是可能被用户传递进来的)。  
为了解决这个问题，你可以创建一个独一无二的私有对象实例，就像上面的_no_value变量那样。 在函数里面，你可以通过检查被传递参数值跟这个实例是否一样来判断。 这里的思路是用户不可能去传递这个_no_value实例作为输入。 因此，这里通过检查这个值就能确定某个参数是否被传递进来了。  
这里对 object() 的使用看上去有点不太常见。object 是python中所有类的基类。 你可以创建 object 类的实例，但是这些实例没什么实际用处，因为它并没有任何有用的方法， 也没有任何实例数据(因为它没有任何的实例字典，你甚至都不能设置任何属性值)。 你唯一能做的就是测试同一性。这个刚好符合我的要求，因为我在函数中就只是需要一个同一性的测试而已。

#### 5.定义匿名或内联函数  
当一些函数很简单，仅仅只是计算一个表达式的值的时候，就可以使用lambda表达式来代替了。

In [5]:
add = lambda x, y: x + y
print(add(2,3))
print(add('hello', 'world'))

5
helloworld


lambda表达式典型的使用场景是排序或数据reduce等：  
尽管lambda表达式允许你定义简单函数，但是它的使用是有限制的。 你只能指定单个表达式，它的值就是最后的返回值。也就是说不能包含其他的语言特性了， 包括多个语句、条件表达式、迭代以及异常处理等等。

In [6]:
names = ['David Beazley', 'Brian Jones','Raymond Hettinger', 'Ned Batchelder']
sorted(names, key=lambda name: name.split()[-1].lower())

['Ned Batchelder', 'David Beazley', 'Raymond Hettinger', 'Brian Jones']

#### 6. 匿名函数捕获变量值

In [8]:
x = 10
a = lambda y: x + y
x = 20
b = lambda y: x + y

#如果你认为结果是20和30，那么你就错了：
print(a(10))
print(b(10))

30
30


这其中的奥妙在于lambda表达式中的x是一个自由变量， 在运行时绑定值，而不是定义时就绑定，这跟函数的默认值参数定义是不同的。

In [9]:
x = 10
a = lambda y, x=x: x + y
x = 20
b = lambda y, x=x: x + y
print(a(10))
print(b(10))

20
30


在这里列出来的问题是新手很容易犯的错误，有些新手可能会不恰当的使用lambda表达式。 比如，通过在一个循环或列表推导中创建一个lambda表达式列表，并期望函数能在定义时就记住每次的迭代值。例如：

In [10]:
funcs = [lambda x: x+n for n in range(5)]
for f in funcs:
    print(f(0))

4
4
4
4
4


In [11]:
funcs = [lambda x, n=n: x+n for n in range(5)]
for f in funcs:
    print(f(0))

0
1
2
3
4


#### 7. 减少可调用对象的参数个数
如果需要减少某个函数的参数个数，你可以使用 functools.partial() 。 partial() 函数允许你给一个或多个参数设置固定的值，减少接下来被调用时的参数个数。

In [12]:
def spam(a, b, c, d):
    print(a, b, c, d)

from functools import partial
s1 = partial(spam, 1) # a = 1
s1(2, 3, 4)
s1(4, 5, 6)

s2 = partial(spam, d=42) # d = 42
s2(1, 2, 3)
s2(4, 5, 5)

s3 = partial(spam, 1, 2, d=42) # a = 1, b = 2, d = 42
s3(3)
s3(4)
s3(5)

1 2 3 4
1 4 5 6
1 2 3 42
4 5 5 42
1 2 3 42
1 2 4 42
1 2 5 42


**第一个例子是，假设你有一个点的列表来表示(x,y)坐标元组。 你可以使用下面的函数来计算两点之间的距离：**

In [13]:
points = [ (1, 2), (3, 4), (5, 6), (7, 8) ]

import math
def distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return math.hypot(x2 - x1, y2 - y1)

#现在假设你想以某个点为基点，根据点和基点之间的距离来排序所有的这些点。
pt = (4, 3)
points.sort(key=partial(distance,pt))
points

[(3, 4), (1, 2), (5, 6), (7, 8)]

更进一步，partial() 通常被用来微调其他库函数所使用的回调函数的参数。 例如，下面是一段代码，使用 `multiprocessing` 来异步计算一个结果值， 然后这个值被传递给一个接受一个result值和一个可选logging参数的回调函数：

In [ ]:
def output_result(result, log=None):
    if log is not None:
        log.debug('Got: %r', result)

# A sample function
def add(x, y):
    return x + y

if __name__ == '__main__':
    import logging
    from multiprocessing import Pool
    from functools import partial

    logging.basicConfig(level=logging.DEBUG)
    log = logging.getLogger('test')

    p = Pool()
    p.apply_async(add, (3, 4), callback=partial(output_result, log=log))
    p.close()
    p.join()

当给 apply_async() 提供回调函数时，通过使用 partial() 传递额外的 logging 参数。 而 multiprocessing 对这些一无所知——它仅仅只是使用单个值来调用回调函数。